In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os

from random import randint, shuffle

import matplotlib.pyplot as plt 
%matplotlib inline

# Prepare Data

In [2]:
import pandas as pd
import cv2

In [3]:
data_path = 'sample/sample/'
annot = pd.read_csv('trainLabels.csv')
annot.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [4]:
info_list = [(data_path+img_name+'.jpeg', level) for img_name, level in zip(annot['image'], annot['level'])]

In [5]:
batchs = info_list[:10]

In [6]:
img2 = cv2.imread(info_list[0][0])
img2.shape

(3168, 4752, 3)

# Configure

In [7]:
train_data = batchs

batch_size = 32
n_epochs = 30 

num_samples_per_epoch = len(train_data)
num_batches_per_epochs = num_samples_per_epoch // batch_size

img_size = 224
channel_n = 3
class_n = 5

print("Image size : ", img_size)
print("%d train images" % (len(train_data)))
# print("%d  test  images" % (len(test_data)))
print("Total Epochs : ", n_epochs)
print("Number of Batches per Epoch : ", num_batches_per_epochs)

Image size :  224
10 train images
Total Epochs :  30
Number of Batches per Epoch :  0


# Model : STL

In [8]:
def pRelu(x, name='P_ReLU'):
    with tf.variable_scope(name):
        alpha = tf.get_variable('a', x.get_shape()[-1], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
        return tf.maximum(0.0, x) + tf.minimum(0.0, alpha*x)

In [9]:
def conv2d(input_, output_n, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=pRelu, name='conv2d'):
    with tf.variable_scope(name):
        W = tf.get_variable('W', [k_h, k_w, input_.get_shape()[-1], output_n], 
                            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable('b', [output_n], initializer=tf.constant_initializer(bias))
        conv = tf.nn.conv2d(input_, W, strides=[1, d_h, d_w, 1], padding='SAME', name='conv')
        conv_b = tf.nn.bias_add(conv, b)
        act = activation_fc(conv_b, 'act')
        
        tf.summary.histogram('Weight', W)
        tf.summary.histogram('Bias', b)
        tf.summary.histogram('Conv_Layer', conv)
        tf.summary.histogram('Conv_with_bias', conv_b)
        tf.summary.histogram('Activation', act)

        return act


def flatten(input_, name='flatten'):
    vec_dim = input_.get_shape()[1:]
    n = vec_dim.num_elements()
    with tf.name_scope(name):
        return tf.reshape(input_, [-1, n])


def linear(input_, output_size, stddev=0.02, bias=0.0, name='linear'):
    input_size = input_.get_shape().as_list()[1]
    with tf.variable_scope(name):
        W = tf.get_variable('W', [input_size, output_size], tf.float32,
                           initializer=tf.random_normal_initializer(stddev=stddev))
        b = tf.get_variable('b', [output_size], initializer=tf.constant_initializer(bias))
        logits = tf.nn.xw_plus_b(input_, W, b, name='logits')

        tf.summary.histogram('Weight', W)
        tf.summary.histogram('Bias', b)
        tf.summary.histogram('logit', logits)

        return logits
    
def drop_out(x, prob, name='drop_out'):
    with tf.variable_scope(name):
        drop_layer = tf.nn.dropout(x, prob)
        tf.summary.histogram('Drop_Out', drop_layer)
        return drop_layer

def stl_cross_enropy(cls_pred, loc_pred, true, a=0.1):
    '''
    :param a: Hyperparameter. default = 0.1 and increased to 0.9 after 60 epochs
             to determine the level of importance between classifier and localizer.
    '''
    cls_softmax = tf.nn.softmax(cls_pred, name='Clssification_Softmax')
    loc_softmax = tf.nn.softmax(loc_pred, name='Localization_Softmax')

    return - (1 - a)*tf.reduce_mean(true * tf.log(cls_softmax)) - a * tf.reduce_mean(true * tf.log(loc_softmax))

def batch_norm(x, epsilon=1e-5, momentum=0.9, train=True, name="batch_norm"):
    with tf.variable_scope(name):
        return tf.contrib.layers.batch_norm(x, decay=momentum,
                                            updates_collections=None, epsilon=epsilon, scale=True, scope=name)

In [10]:
class STL(object):
    def __init__(self):
        self.img_size = img_size
        self.class_n = class_n
        self.default_act_fn = pRelu
        print("Self Transfer Learning is Ready")
    
    def conv_layer(self, input_):
        self.conv1 = conv2d(input_, output_n=96, k_h=11, k_w=11, d_h=4, d_w=4, bias=0.0, activation_fc=self.default_act_fn, name='Conv1')
        self.pool1 = tf.nn.max_pool(self.conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='Pool1')
        self.norm1 = batch_norm(self.pool1, epsilon=1e-5, momentum=0.9, train=True, name="Batch_Norm1")
        
        self.conv2 = conv2d(self.norm1, output_n=256, k_h=5, k_w=5, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv2')
        self.pool2 = tf.nn.max_pool(self.conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='Pool2')
        self.norm2 = batch_norm(self.pool2, epsilon=1e-5, momentum=0.9, train=True, name="Batch_Norm2")
        
        self.conv3 = conv2d(self.norm2, output_n=384, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv3')

        self.conv4 = conv2d(self.conv3, output_n=384, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv4')
        
        self.conv5 = conv2d(self.conv4, output_n=256, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv5')
        self.pool5 = tf.nn.max_pool(self.conv5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='Pool3')
        
        return self.pool5
        
        
    def classifier(self, input_, keepratio):
        self.flat1 = flatten(input_, name='flatten')
        self.line1 = linear(self.flat1, 4096, name='fc_layer_4096')
        self.drop2 = drop_out(self.line1, prob=keepratio)
        self.line2 = linear(self.drop2, 4096, name='fc_layer_1000')
        self.drop2 = drop_out(self.line2, prob=keepratio)
        self.line3 = linear(self.drop2, self.class_n, name='fc_layer_'+str(self.class_n))
        return self.line3
    
    def localizer(self, input_, alpha):
        # Weight : Gaussian dist with standard deviation 0.01, and biases = 0
        self.conv6 = conv2d(input_, output_n = 2048, k_h=1, k_w=1, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv6')
        self.gap = tf.reduce_mean( self.conv6, [1,2] )
        with tf.variable_scope("GAP"):
            self.gap_w = tf.get_variable("W", shape=[2048, self.class_n],initializer=tf.random_normal_initializer(0., 0.01))

        return tf.matmul(self.gap, self.gap_w)


    def get_classmap(self, conv6, label): 
        conv6_resized = tf.image.resize_bilinear( conv6, [self.img_size, self.img_size] )
        with tf.variable_scope("GAP", reuse=True):
            label_w = tf.gather(tf.transpose(tf.get_variable("W")), label)
            label_w = tf.reshape( label_w, [-1, 2048, 1] )

        conv6_resized = tf.reshape(conv6_resized, [-1, self.img_size*self.img_size, 2048]) 

        classmap = tf.batch_matmul( conv6_resized, label_w )
        classmap = tf.reshape( classmap, [-1, self.img_size, self.img_size] )
        
        return classmap

    
    def inference(self, input_, keepratio, alpha):
        conv_layers = self.conv_layer(input_)
        cls_pred = self.classifier(conv_layers, keepratio)
        loc_pred = self.localizer(conv_layers, alpha)
        return cls_pred, loc_pred
    

In [11]:
# Test 용 지워야 한다. 
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, channel_n])
y = tf.placeholder(tf.int64, shape=[None, 5])
alpha = tf.placeholder(tf.float32)
keepratio = tf.placeholder(tf.float32) 

# Image Ready

In [12]:
def read_img(img, transform = True):
    if transform == True:
        
        '''
        # Cropping Image 
        row_range = img.shape[0]-img_size
        col_range = img.shape[1]-img_size

        random_row = randint(0, row_range)
        random_col = randint(0, col_range)

        img = img[random_row:random_row+img_size, random_col:random_col+img_size]
        '''
        
        # Rotation
        rot_deg = randint(0,3) * 90

        rot_mat = cv2.getRotationMatrix2D((img.shape[0]//2, img.shape[1]//2), rot_deg, 1)
        img = cv2.warpAffine(img, rot_mat, img.shape,flags=cv2.INTER_LINEAR)


        # Flipping Image
        flip = randint(0,1)
        if flip == 1:
            img = cv2.flip(img,1)
        
    img = cv2.resize(img, (img_size, img_size)).reshape(img_size, img_size, channel_n)
    
    return img

def make_batch(batchs, transform = True):
    for n, (img_path, level) in enumerate(batchs):
        img = cv2.imread(img_path)
        img = read_img(img, transform)
        label = [0] * 5
        label[int(level)] = 1

        if n == 0:
            img_batch = [img]
            label_batch = [label]
        else:
            img_batch = np.concatenate((img_batch, [img]))
            label_batch = np.concatenate((label_batch, [label]))

    return img_batch, label_batch

# Optimization

In [15]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, channel_n])
y = tf.placeholder(tf.float32, shape=[None, class_n])
alpha = tf.placeholder(tf.float32)
keepratio = tf.placeholder(tf.float32)

In [16]:
model = STL()
cls_pred, loc_pred = model.inference(x, keepratio, alpha)

Self Transfer Learning is Ready


In [17]:
cost = stl_cross_enropy(cls_pred, loc_pred, y, alpha)   # alpha : 처음엔 0.1로 시작하고서, 60 epochs 후에는 0.9

In [18]:
initial_learning_rate = 0.001
learning_rate_decay_factor = 0.1
num_epochs_per_decay = 20


global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
var_list = tf.trainable_variables()
decay_steps = int(num_batches_per_epochs * num_epochs_per_decay)
lr = tf.train.exponential_decay(initial_learning_rate,
                                global_step,
                                decay_steps,
                                learning_rate_decay_factor,
                                staircase=True)

optimizer = tf.train.AdamOptimizer(lr)
grads_and_vars = optimizer.compute_gradients(cost, var_list=var_list)

In [19]:
pred = cls_pred + loc_pred
corr = tf.equal(tf.argmax(y,1), tf.argmax(pred, 1)) 
accr = tf.reduce_mean(tf.cast(corr, "float"))
optm = optimizer.apply_gradients(grads_and_vars, global_step)
print ("Functions ready")

Functions ready


In [22]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

from datetime import datetime

In [ ]:
weight_path = 'data/'
model_path = 'model/retina/STL_AlexNet'

if not os.path.exists(trainset_path):
    


# Start Training

In [ ]:
start_time = datetime.now()
print("Start Training at ", start_time)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()

    for epoch in range(n_epochs):
        trainLoss = []; trainAcc = []
        np.random.shuffle(train_data)
        
        if epoch < 60:
            a = 0.1
        else:
            a = 0.9
        
        for batch_idx in range(num_batches_per_epochs):
            batch_files = train_data[batch_idx * batch_size:(batch_idx + 1) * batch_size]
            
            train_imgBatch, train_labelBatch = make_batch(train_data, transform = True)
            
            train_feed = {x: train_imgBatch, y: train_labelBatch, keepratio:0.7, alpha:a}
            sess.run(optm, train_feed)
            
            trainLoss.append(sess.run(cost, train_feed))
            trainAcc.append(sess.run(accr, train_feed))

        
        # Average loss and accuracy
        trainLoss = np.mean(trainLoss); trainAcc = np.mean(trainAcc)
        
        np.random.shuffle(test_data)
        test_batch = test_data[:batch_size]
        
        test_imgBatch, test_labelBatch = make_batch(test_data, transform = False)
        
        test_feed = {x: test_imgBatch, y: test_labelBatch}
        valAcc = sess.run(pred, test_feed)
        
        if epoch == 0:
            print("Expected During Time : ", (datetime.now()-start_time)*n_epochs)
            print("==============================================================================")
        
        print ("[%02d/%02d] trainLoss: %.4f trainAcc: %.2f valAcc: %.2f \n" 
               % (epoch, n_epochs, trainLoss, trainAcc, valAcc))


print("Training Finished at ", datetime.now())
print("During Time : ", datetime.now() - start_time)

# Get Test

In [ ]:
c1,c2,c3,c4,conv5, conv6, gap, output = detector.inference( images_tf )
classmap = detector.get_classmap( labels_tf, conv6 )

sess = tf.InteractiveSession()
saver = tf.train.Saver()

saver.restore( sess, model_path )

for start, end in zip(
    range( 0, len(testset)+batch_size, batch_size),
    range(batch_size, len(testset)+batch_size, batch_size)):

    current_data = testset[start:end]
    current_image_paths = current_data['image_path'].values
    current_images = np.array(map(lambda x: load_image(x), current_image_paths))

    good_index = np.array(map(lambda x: x is not None, current_images))

    current_data = current_data[good_index]
    current_image_paths = current_image_paths[good_index]
    current_images = np.stack(current_images[good_index])
    current_labels = current_data['label'].values
    current_label_names = current_data['label_name'].values

    conv6_val, output_val = sess.run(
            [conv6, output],
            feed_dict={
                images_tf: current_images
                })

    label_predictions = output_val.argmax( axis=1 )
    acc = (label_predictions == current_labels).sum()

    classmap_vals = sess.run(
            classmap,
            feed_dict={
                labels_tf: label_predictions,
                conv6: conv6_val
                })

    classmap_answer = sess.run(
            classmap,
            feed_dict={
                labels_tf: current_labels,
                conv6: conv6_val
                })

    classmap_vis = map(lambda x: ((x-x.min())/(x.max()-x.min())), classmap_answer)

    for vis, ori,ori_path, l_name in zip(classmap_vis, current_images, current_image_paths, current_label_names):
        print l_name
        plt.imshow( ori )
        plt.imshow( vis, cmap=plt.cm.jet, alpha=0.5, interpolation='nearest' )
        plt.show()